# Select the most difficult images to label next

*This notebook shows you how to plug your ML model in Encord Active to rank the images according to an acquisition function.
We assume that you already have installed encord-active in your local environment, if not please visit [here](https://docs.encord.com/active/docs/installation).*

### The  Motivation

Let's say you have an image dataset consist of different airplanes. You want to train an object detector model to get the bounding boxes of the planes and later crop those regions to create a collage of airplanes pictures for yourself.
[Here](https://storage.googleapis.com/encord-active-notebook-demos/airplanes-active-learning-demo.zip), we have an Encord Active project like this. Download the zipped folder, extract it, and run encord active in the project folder or its parent folder to visualize this project.

To start the Encord-Active app, run the following CLI command:

```shell
encord-active visualize
```

You can visualize the images according to certain image-level metrics in the Data Quality -> Explorer page.

![airplane images](images/explorer-airplanes.png)

There are 799 images in the dataset; however, you do not want to label all of them, which would be quite cumbersome. You only want to label 50 images and you want to select the best 50 images that will give you the best outcome in terms of model performance.

In an active learning workflow, you generally have a model trained on a few initial labeled samples. In this notebook, for the simplicity, we will use a HuggingFace YOLOS-tiny model that is trained on COCO dataset, in which there is an `airplane` class! So, we will use the YOLOS-tiny model's output for the `airplane` class to rank the images.

As an acquisition function, you will use Encord Active's `AverageFrameScore`. This acquisition function gets the average confidence scores of the predictions in each image, if there is no prediction, it assigns a value of zero. You think that in all of your images there is at least one airplane, so this acquisition function is very suitable for your task. If the score is low, model is having difficulty in detecting airplanes in those images, so you may want to collect these samples to label next.

Let's start, you will see how easy to do that with Encord Active!

## 1. Install the Huggingface transformer library for the object detecion model into your local environment

If there is any running Encord Active app, please terminate it bofore running the following commands


In [ ]:
! pip install transformers

## 2. Import the necessary packages

In [17]:
from pathlib import Path
from typing import List
from PIL import Image
import numpy as np
from transformers import YolosImageProcessor, YolosForObjectDetection
import torch
from encord_active.lib.metrics.acquisition_metrics.common import (
    BaseObjectModel,
    BoundingBoxPrediction,
)
from encord_active.lib.metrics.acquisition_metrics.acquisition_functions import MeanObjectConfidence
from encord_active.lib.metrics.execute import execute_metrics
from encord_active.lib.project.project_file_structure import ProjectFileStructure
from encord_active.lib.metrics.metadata import fetch_metrics_meta
from encord_active.lib.metrics.io import  get_metric_metadata
from encord_active.lib.metrics.metadata import update_metrics_meta

## 3. Implement a BaseObjectModel class

This class will inherit the `BaseObjectModel` class and only  implement the `predict_objects()` method, where a Pillow's Image is taken as an argument and list of bounding boxes along with their predictions will be returned.

In the below code, we first take the logits for each bounding box prediction and calculate the confidences for each class using softmax. There are 92 output values for each bounding box; id=0 refers to background class, id=1-90 referst to COCO classes, and id=91 indicates the probability of `no object`. In this example, we only include the predictions, whose `no object` probability is less than 0.5 (i.e., being an object probability is greater than 0.5). Id 5 refers to the `airplane` class, which we will use to send its probability to the acquisition function.

In [18]:
class HuggingFaceYolosTinyModel(BaseObjectModel):
    """
    For more information on how to use this model, check here: https://huggingface.co/hustvl/yolos-tiny
    """
    def __init__(self):
        self.processor = YolosImageProcessor.from_pretrained("hustvl/yolos-tiny")
        self.model = YolosForObjectDetection.from_pretrained('hustvl/yolos-tiny')

    @torch.no_grad()
    def predict_objects(self, image: Image)-> List[BoundingBoxPrediction]:
        if image.mode != 'RGB':
            image = image.convert('RGB')

        data = self.processor(images=image, return_tensors="pt")
        outputs = self.model(**data)
        output_probs = outputs.logits[0].softmax(dim=1)

        output: List[BoundingBoxPrediction] = []

        # Index 91 refers to the probability of `no object`, here we only collect the
        # predictions that have a high probability of being an object. Index 5 refers to the `airplane` class
        for row_id in np.where(output_probs[:, 91] < 0.5)[0]:
            box = outputs.pred_boxes[0, row_id].numpy()
            output.append(BoundingBoxPrediction(x=box[0], y=box[1], w=box[2] - box[0], h=box[3] - box[1],
                                                class_probs=np.array([output_probs[row_id, 5].item(), 1 - output_probs[row_id, 5].item()])))

        return output

## 4. Run the acquisition metric

`use_cache_only=True` provides using the data in your local system and `skip_labeled_data=True` enables skipping the data that already is already annotated.

In [ ]:
project_fs = ProjectFileStructure(Path("/path/to/airplanes-project"))

# Create the acquisition metric
acq_func = MeanObjectConfidence(HuggingFaceYolosTinyModel())

# Run the acquisition metric
execute_metrics([acq_func], data_dir=project_fs.project_dir, use_cache_only=True, skip_labeled_data=True)

# Update the metric files
metrics_meta = fetch_metrics_meta(project_fs)
metrics_meta[acq_func.metadata.title]= get_metric_metadata(acq_func)
update_metrics_meta(project_fs, metrics_meta)

project_fs.db.unlink(missing_ok=True)

## 5. Next steps

Now, we have run our acquisition function on all samples. When you open the project from the start, you will see this new acquisition metric in the Data Quality Explorer page. Click the arrow in the metrics dropdown list and select 'Mean Object Confidence'. The images that have low scores mean that model is not confident in its predictions, so they should have higher priority when labelling.

![airplane images sorted](images/mean-object-confidence-explorer.png)

As a next step you have several options now; you can select the first N samples and:

1. create a new project on Encord Annotate platform to label.
2. export them using the Actions tab and use them in your own annotation tool.

This was a local Encord Active project; however, you may perform the same operations easily for your Encord Annotate project once you import them to your local environment. After importing the Encord Annotate project, if you update your encord annotate project by labeling samples, and want to run this acquisition function again on the rest unlabeled images, you can use the following CLI command to synchronize your local project with the Encord Annotate project:

```shell
encord-active refresh
```

Now, you can run the above scripts again to update the acquisition function metric.